In [1]:
%matplotlib widget
import os,sys
import csv
import copy
import math
import collections
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from clawpack import pyclaw
from clawpack import riemann
import numpy as np
from scipy.integrate import simps, trapz
from scipy.interpolate import interp1d

sys.path.append("/pymorph")
from schemes.weno import get_left_flux,get_right_flux
from schemes.weno import get_stencil
import sediment_transport.sed_trans as sedtrans

from models.shallow_water_solver import shallow_water_solver
from schemes.avalanche_scheme import *

from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D

from IPython.display import display, Math, Latex


%load_ext autoreload
%autoreload 2

plt.style.use(['dark_background'])

In [2]:
profile_folder = '/data/PhD/A3DOWN 2006-06-09 (S-800, 30-15cm)'

In [3]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    


def readWiebeFile(filepath, resolution = 0.5):
    xs = []
    zs = []
    with open(filepath) as f:
        for line in f:
            values = line.split(',')
            if is_number(values[0]) and is_number(values[1]):
                x = float(values[0])/100.
                z = float(values[1])/100.
                xs.append(x)
                zs.append(z)
                
    xs = np.array(xs)
    zs = np.array(zs)
    
    #--------------------------------
    # Increase the resolution on the grid
    #--------------------------------
    xmax = 12.0 #xs.max()
    nx = len(xs)
    f = interp1d(xs, zs)
    
    xnew = np.linspace(0., xmax, num=800)
    znew = f(xnew)
    return xnew, znew


xprofile = None
zdict = {}

for filename in os.listdir(profile_folder):
    name = filename.split('.')[0]
    profile = None
    if len(name)==10:
        profile = '0{0}'.format(name[-1:])
    else:
        profile = name[-2:]
    
    x, z = readWiebeFile(os.path.join(profile_folder,filename), resolution=0.25)
    
    xprofile = x
    zdict[profile] = z

profileDf = pd.DataFrame(zdict)

profileDf = profileDf.reindex(sorted(profileDf.columns), axis=1)

In [4]:
plt.plot(xprofile,profileDf['00'])
plt.show()

FigureCanvasNbAgg()

In [5]:
xprofile[1]-xprofile[0]

0.015018773466833541

In [6]:
#--------------------------------
# Set up the constants
#--------------------------------
nP=0.4
Slope = 1.26/1000 # Based on table 4.1 in Wiebe
mannings = 0.022
water_surface= 0.20      # Surface water elevation
beta = 0.01

starting_profile = '02'

# Sediment properties
D50 = 0.001     # Particle Diameter
rho_particule = 2650


nx = len(xprofile)
xmax = xprofile.max()
#--------------------------------
# Set up the domain, BCs and ICs
#--------------------------------
import logging
logger = logging.getLogger('pyclaw')
logger.setLevel(logging.CRITICAL)

x = pyclaw.Dimension(0.0,xmax,len(xprofile),name='x')
domain = pyclaw.Domain(x)
state = pyclaw.State(domain,1)
xc = state.grid.x.centers
dx = state.grid.delta[0]
print('Grid dx = {0}'.format(dx))
print('Grid nx = {0}'.format(len(xc)))

zc = np.array(profileDf[starting_profile].tolist().copy())
z_hydro = np.array(profileDf[starting_profile].tolist().copy())

 # Gravitational constant
state.problem_data['grav'] = 9.81
state.problem_data['sea_level'] = 0.0


# Set up the inital state of the flow
shw_slvr = shallow_solver(domain, slope=Slope, mannings=mannings)
u, surf,h = shw_slvr.run(zc,water_surface,3000)


if np.isnan(u).any():
    # Get the flow
    print( u)
    raise ValueError('Failed to calculate the flow')


Grid dx = 0.015
Grid nx = 800


NameError: name 'shallow_solver' is not defined

In [7]:
fig = plt.figure(figsize=(12, 4))
plt.plot(xc,surf)
plt.plot(xc,zc,'r')

FigureCanvasNbAgg()

In [8]:

#--------------------------------
# Initialize the sed transport 
#--------------------------------    
# Eq 57

qbedload = np.zeros(nx)
roe_speed = np.zeros(nx)
flux = np.zeros(nx)

#--------------------------------
#  Setup the model run parameters
#--------------------------------
simTime = 1800. # Runtime in seconds
dt = 1 # Time step size (s)
nt = int(simTime/dt)  # Number of time steps


#--------------------------------
# Set up the model reporting parameters
#--------------------------------
z0 = zc.copy() 
z600 = zc.copy() 
z2000 = zc.copy()
verts = []
extractionInterval = 120.
tsteps=[]
iterations = 0
cntr=0
#--------------------------------
#  Run the model
#--------------------------------
for n in range(1,nt):  
    zn = zc.copy() 
    for i in range(0,nx): #i=2
        zloc = get_stencil(zn,i-2,i+4)        
        # Since k=3
        # stencil is i-2 to i+2 
        qloc = get_stencil(qbedload,i-2,i+4)
        if len(qloc)!=6:
            raise ValueError('Stencil is incorrect')
 
        # Determine the Upwind flux
        # The 0.5 comes from the c+abs(c) which is 2 if the wave speed is +ive
        # this is the evaluation of the left and right based fluxes. Eq. 18 and 19        
        if (zloc[3]-zloc[2]) == 0.0:
            roe_speed[i]=np.sign( (qloc[3]-qloc[2]) )
        else:
            roe_speed[i]=np.sign( (qloc[3]-qloc[2])/ (zloc[3]-zloc[2]) )
        
        if roe_speed[i] >= 0.0:
            flux[i] = get_left_flux(qloc)
        else:
            flux[i] = get_right_flux(qloc)
        
    # Need the sign of the phase speed
    # Need to check this out
    for i in range(0,nx): #i=2       
        floc = get_stencil(flux,i-1,i+1)
        zc[i] = zn[i]-(1./(1.-nP))*dt/dx*(floc[1]-floc[0])
    
    bed_max_delta = np.max(np.abs(zn - zc))
    
    # Apply the avalanche model
    zc, iterations1 = avalanche_model(dx,xc, zc, adjustment_angle=28.)
    # Now flip it to run in reverse
    zflip = np.flip(zc, axis=0)
    zflip, iterations1 = avalanche_model(dx,xc, zflip, adjustment_angle=28.)
    zc = np.flip(zflip, axis=0)
    
    #zc = z_updated.copy()
    
    # Re-run the solver and updated the sediment transport
    # Check if the avalanche model was updated OR the bed changed greater than the threshold
    #if iterations > 10 or bed_max_delta > 0.00005:
    iterations+=1
    if iterations > 2:
        u, surf,h = shw_slvr.run(zc,water_surface, 120)
        iterations = 0

    slope = np.gradient(zc,dx)
    
    ## Nov 13 2018 - Can modify later
    for i in range(0,nx): #i=2        
        #zlocal = get_stencil(zc,i-2,i+2)
        #qbedload[i] = sedtrans.get_unit_bed_load2(h[i],u[i],D50,rho_particule,type='bagnold')
        qbedload[i] = sedtrans.get_unit_bed_load_slope(h[i],u[i],D50,slope[i], 
                                                       rho_particule, 
                                                       angleReposeDegrees = 30.0, 
                                                       type='bagnold',
                                                        useSlopeAdjust=False)

    
    if (n*dt / extractionInterval) == math.floor(n*dt / extractionInterval):
        verts.append(list(zip(xc.copy(),zc.copy())))
        tsteps.append(n*dt)        
        np.save('flume_dunes', verts)
    
    cntr += 1
    if cntr == extractionInterval:
        cntr = 0
        print('Time step {0} of {1} - bed_max_delta {2}'.format(n,nt,bed_max_delta))    
        
print('Done')

2019-04-09 23:01:32,127 INFO CLAW: Solution 0 computed for time t=0.000000
2019-04-09 23:01:45,010 INFO CLAW: Solution 1 computed for time t=70.921507
2019-04-09 23:01:45,281 INFO CLAW: Solution 0 computed for time t=0.000000
2019-04-09 23:01:57,099 INFO CLAW: Solution 1 computed for time t=70.921468
2019-04-09 23:01:57,295 INFO CLAW: Solution 0 computed for time t=0.000000
2019-04-09 23:02:09,141 INFO CLAW: Solution 1 computed for time t=70.921471
2019-04-09 23:02:09,435 INFO CLAW: Solution 0 computed for time t=0.000000
2019-04-09 23:02:21,561 INFO CLAW: Solution 1 computed for time t=70.921373
2019-04-09 23:02:21,816 INFO CLAW: Solution 0 computed for time t=0.000000
2019-04-09 23:02:33,329 INFO CLAW: Solution 1 computed for time t=70.921371
2019-04-09 23:02:33,601 INFO CLAW: Solution 0 computed for time t=0.000000
2019-04-09 23:02:45,256 INFO CLAW: Solution 1 computed for time t=70.921392
2019-04-09 23:02:45,513 INFO CLAW: Solution 0 computed for time t=0.000000
2019-04-09 23:02:57

In [13]:
fig = plt.figure(figsize=(14, 4))
plt.plot(xc,surf)
plt.plot(xc,profileDf[starting_profile],'g')
plt.plot(xc,zc,'r')
#plt.plot(xc,profileDf['04'],'c')
#plt.plot(xc,profileDf['05'],'c')
plt.plot(xc,profileDf['06'],'c')

FigureCanvasNbAgg()

In [10]:
u.mean()

0.5047756045946366

In [11]:
xc[1] - xc[0]

0.015